(ignore the .md proposal file we need to submit a .ipynb and html file instead)

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

# Heart Disease

***Introduction***

...

***Preliminary Exploratory Data Analysis***

Reading the file into R from the web:

(Dataset and information from https://www.kaggle.com/cherngs/heart-disease-cleveland-uci and https://archive.ics.uci.edu/ml/datasets/Heart+Disease)

> age = age in years
>
> sex = 0: female; 1: male
>
> cp = type of chest pain;
>
> - 0: typical angina
>    
> - 1: atypical angina
>    
> - 2: non-anginal pain
>    
> - 3: asymptomatic
>
> trestbps = resting blood pressure
>
> chol = serum cholesterol in mg/dL
>
> fbs = fasting blood sugar > 120 mg/dL (0: false; 1: true)
>
> restecg = resting electrocardiographic results
>
> - 0: normal
>
> - 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05mV)
>
> - 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
>
> thalach = maximum heart rate achieved
>
> exang = exercise induced angina (0: no; 1: yes)
>
> oldpeak = ST depression induced by exercise relative to rest
>
> slope = slope of the peak exercise ST segment
>
> - 0: upsloping
>
> - 1: flat
>
> - 2: downsloping
>
> ca = number of major vessels (0-3) coloured by fluoroscopy
>
> thal = thalassemia
>
> - 0: normal
>
> - 1: fixed defect
>
> - 2: reversible defect
>
> condition = disease condition
>
> - 0: no disease
>
> - 1: disease

In [3]:
heart_data <- read_csv("heart_cleveland_upload.csv")
heart_data

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restecg = col_double(),
  thalach = col_double(),
  exang = col_double(),
  oldpeak = col_double(),
  slope = col_double(),
  ca = col_double(),
  thal = col_double(),
  condition = col_double()
)



age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
64,1,0,110,211,0,2,144,1,1.8,1,0,0,0
64,1,0,170,227,0,2,155,0,0.6,1,0,2,0
63,1,0,145,233,1,2,150,0,2.3,2,0,1,0
61,1,0,134,234,0,0,145,0,2.6,1,2,0,1
60,0,0,150,240,0,0,171,0,0.9,0,0,0,0
